In [42]:
import numpy as np
import matplotlib.pyplot as plt
import math

In [59]:
class Problema(object):
    pass

class BuscaLocal(object):
    pass

class Estado(object):
    def __init__(self, valor):
        self.valor = valor

class EstadoPSVM(Estado):
    def __init__(self, valor, e, d):
        super(EstadoPSVM, self).__init__(valor)
        self.e = e
        self.d = d
    
    def __str__(self):
        return 'E=%d, D=%d, S=%s' % (self.e, self.d, self.valor)
        
class ProblemaSubVetorMaximo(Problema):
    
    @classmethod
    def from_aleatorio(cls, n_elementos, e_inicial=None, d_inicial=None):
        cls.vetor = np.random.randint(-20,20,(n_elementos))
        if not e_inicial:
            e = np.random.randint(0, n_elementos-1)
            d = np.random.randint(e, n_elementos)
        else:
            e = e_inicial
            d = d_inicial
        valor_ini = ProblemaSubVetorMaximo.__valor_solucao(cls.vetor,e,d)
        cls.estado_inicial = EstadoPSVM(valor_ini, e, d)
        return cls()
    
    @classmethod
    def from_vetor(cls, vetor, e_inicial, d_inicial):
        cls.vetor = vetor
        cls.estado_inicial = EstadoPSVM(ProblemaSubVetorMaximo.__valor_solucao(vetor,e_inicial, d_inicial), e_inicial, d_inicial)
        return cls()
        
    @staticmethod
    def __valor_solucao(v, e, d):
        return np.sum(v[e:d+1])
    
    def valor_solucao(self, e, d):
        return ProblemaSubVetorMaximo.__valor_solucao(self.vetor, e, d)
    
    def vizinhos(self, estado):
        vizs = []
        e = estado.e
        d = estado.d
        for i in [e-1,e,e+1]:
            if (i < 0) or (i >= self.vetor.shape[0]):
                continue
            for j in [d-1,d,d+1]:
                if (j < 0) or (j >= self.vetor.shape[0]):
                    continue
                if (e == i) and (d == j):
                    continue
                vizs.append(EstadoPSVM(self.valor_solucao(i,j), i, j))
        return vizs
    
    def sol_otima(self):
        #Find maximum subarray (Cormen)
        def fmsa(V, e, d):
            #print (e, d)
            if e == d:
                return (e, d, V[e])
            else:
                meio = int(math.floor((e + d) / 2))
                
                eb, ea, es = fmsa(V, e, meio)
                db, da, ds = fmsa(V, meio + 1, d)
                cb, ca, cs = fmcsa(V, e, meio, d)
                if (es >= ds) and (es >= cs):
                    return (eb, ea, es)
                elif (ds >= es) and (ds > cs):
                    return (db, da, ds)
                else:
                    return (cb, ca, cs)
        
        def fmcsa(V, e, m, d):
            es = -np.infty
            s = 0
            for i in range(m, e-1, -1):
                s += V[i]
                if s > es:
                    es = s
                    em = i
            ds = -np.infty
            s = 0
            for j in range(m+1, d+1):
                s = s + V[j]
                if s > ds:
                    ds = s
                    dm = j
            return (em, dm, es + ds)
                    
        solucao = fmsa(self.vetor, 0, self.vetor.shape[0]-1)
            
        return EstadoPSVM(solucao[2], solucao[0], solucao[1])
        
class HillClimbingBL(object):
    
    def __init__(self):
        pass
    
    def buscar(self, problema):
        atual = problema.estado_inicial
        it = 0
        while True:
            print('###Iteração %d###' % it)
            vzs = problema.vizinhos(atual)
            print_vizinhos(vzs)
            vizinho = max(vzs, key=lambda x: x.valor)
            print('Vizinho escolhido: %s' % vizinho)
            
            if vizinho.valor <= atual.valor:
                return atual
            atual = vizinho
            print('Estado atual: %s' % atual)
            print('')
            it+=1

def print_vizinhos(estados):
    print('Vizinhos')
    for e in estados:
        print('\t%s' % (e))
            
#p = ProblemaSubVetorMaximo.from_aleatorio(10)

#v = np.array([13, -3, -25, 20, -3, -16, -23, 18, 20, -7, 12, -5, -22, 15, -4, 7])
#p = ProblemaSubVetorMaximo.from_vetor(v, 5, 12)

p = ProblemaSubVetorMaximo.from_aleatorio(1000)

hc = HillClimbingBL()

print(p.vetor)
print ('Estado Inicial: %s' % p.estado_inicial)

solucao = hc.buscar(p)

print("Solução: %s" % solucao)

sol_otima = p.sol_otima()
print ('Solução ótima: %s' % sol_otima)

[-12 -15 -20  18 -17  19 -17  -1  -4   1  -3  -5 -17   1   3  -1  -1  14
 -20 -17   6   4   1  -7 -20  16  12 -14   0  -3  -6  -8   2 -16  16   1
   0  10  12   2  16  11  12 -15  18 -19   1  -1  12   3 -19 -12  -4  19
 -20  16  18  -6   6   9  -3  -4 -13  14  12 -12 -18  16  14  16   5   1
  16 -11   5  -5 -16 -10   0 -20 -18  11  -6  -5  16  -5  -6   2  13   9
  17  12  -6  -5  -4  13 -15  14  -3 -18  -9  -2  -7 -12  10   4 -10  16
  -6  -4 -10  12  -5 -16 -17  17  -6   5 -11 -16 -19 -17  15  -9 -14  -6
  18  -8  -7 -20  11   6  17   7 -18   1   0   1 -18 -19  15 -12  -8   0
   9  -9   4   6  -6 -12   6  -7 -18  -6  -8  11 -13   7 -18  15  -2 -11
   6  17 -10   9 -17  14 -13 -11 -10  -7 -11  11  13   1 -11  -6  -7  -7
   2  12   2  13  13   6  14 -14  -7  10  11  11 -20  -2  15   1   3   2
   8 -17 -17  -9  15  -6 -13 -10   0 -15  -7  -9 -20  11   2  11   5  -7
  16  -5 -16  -5  -2   4  19  -1   9  11  16  15  16 -16 -20 -11  -9  -9
   6   1   2  11 -18 -14 -18 -16   0 -17  15  -1 -1